In [46]:
from lasagne.layers import InputLayer
from lasagne.layers import DenseLayer
from lasagne.layers import NonlinearityLayer
from lasagne.layers import DropoutLayer
from lasagne.layers import Pool2DLayer as PoolLayer
from lasagne.layers.dnn import Conv2DDNNLayer as ConvLayer
from lasagne.nonlinearities import softmax
from theano import function
from scipy.misc import imread, imresize
import numpy as np
import theano.tensor as T
import matplotlib.pyplot as plt
import os
import pickle
import lasagne
%matplotlib inline

def build_model(input_shape):
    net = {}
    
    net['input'] = InputLayer((None, input_shape[2], input_shape[0], input_shape[1]))
    
    net['conv1_1'] = ConvLayer(net['input'], 64, 3, pad=1, flip_filters=False)
    net['conv1_2'] = ConvLayer(net['conv1_1'], 64, 3, pad=1, flip_filters=False)
    net['pool1'] = PoolLayer(net['conv1_2'], 2)
    net['conv2_1'] = ConvLayer(net['pool1'], 128, 3, pad=1, flip_filters=False)
    net['conv2_2'] = ConvLayer(net['conv2_1'], 128, 3, pad=1, flip_filters=False)
    net['pool2'] = PoolLayer(net['conv2_2'], 2)
    net['conv3_1'] = ConvLayer(net['pool2'], 256, 3, pad=1, flip_filters=False)
    net['conv3_2'] = ConvLayer(net['conv3_1'], 256, 3, pad=1, flip_filters=False)
    net['conv3_3'] = ConvLayer(net['conv3_2'], 256, 3, pad=1, flip_filters=False)
    net['conv3_4'] = ConvLayer(net['conv3_3'], 256, 3, pad=1, flip_filters=False)
    net['pool3'] = PoolLayer(net['conv3_4'], 2)
    net['conv4_1'] = ConvLayer(net['pool3'], 512, 3, pad=1, flip_filters=False)
    net['conv4_2'] = ConvLayer(net['conv4_1'], 512, 3, pad=1, flip_filters=False)
    net['conv4_3'] = ConvLayer(net['conv4_2'], 512, 3, pad=1, flip_filters=False)
    net['conv4_4'] = ConvLayer(net['conv4_3'], 512, 3, pad=1, flip_filters=False)
    net['pool4'] = PoolLayer(net['conv4_4'], 2)
    net['conv5_1'] = ConvLayer(net['pool4'], 512, 3, pad=1, flip_filters=False)
    net['conv5_2'] = ConvLayer(net['conv5_1'], 512, 3, pad=1, flip_filters=False)
    net['conv5_3'] = ConvLayer(net['conv5_2'], 512, 3, pad=1, flip_filters=False)
    net['conv5_4'] = ConvLayer(net['conv5_3'], 512, 3, pad=1, flip_filters=False)
    net['pool5'] = PoolLayer(net['conv5_4'], 2)
    
    
    net['fc6'] = DenseLayer(net['pool5'], num_units=4096)
    net['fc6_dropout'] = DropoutLayer(net['fc6'], p=0.5)
    net['fc7'] = DenseLayer(net['fc6_dropout'], num_units=4096)
    net['fc7_dropout'] = DropoutLayer(net['fc7'], p=0.5)
    net['fc8'] = DenseLayer(net['fc7_dropout'], num_units=1000, nonlinearity=None)
    net['prob'] = NonlinearityLayer(net['fc8'], softmax)

    return net

def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [47]:
with open('vgg19.pkl', 'rb') as f:
    params = pickle.load(f)

In [48]:
def proportional_resize(img, num_pixel = 50000):
    h, w = img.shape[:2]
    k = np.sqrt(float(num_pixel) / (h * w))
    
    return imresize(img, (int(k * h), int(k * w)))

In [49]:
images_path = 'citiscapes/images/'
first_img_path = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(images_path)) for f in fn][0]
img = imread(first_img_path)
first_img = proportional_resize(img)
net = build_model(first_img.shape)

In [50]:
last_vgg_layer = net['pool5']
lasagne.layers.set_all_param_values(last_vgg_layer, params['param values'][:32])

In [51]:
input_X = T.tensor4('X')

output = lasagne.layers.get_output(last_vgg_layer, input_X, deterministic = True)
features = function([input_X], output)

In [52]:
MEAN_VALUES = np.array([104, 117, 123], dtype=np.float32)
 
def preprocess(img):
    result = np.zeros((1, img.shape[2], img.shape[0], img.shape[1]))
   
    result[0, 0, :, :] = img[:, :, 2] - MEAN_VALUES[0]
    result[0, 1, :, :] = img[:, :, 1] - MEAN_VALUES[1]
    result[0, 2, :, :] = img[:, :, 0] - MEAN_VALUES[2]
   
    return result.astype(np.float32)

def deprocess(img):
    result = np.zeros((img.shape[0], img.shape[1], img.shape[2]))
   
    result[:, :, 0] = img[0, 2, :, :] + MEAN_VALUES[2]
    result[:, :, 1] = img[0, 1, :, :] + MEAN_VALUES[1]
    result[:, :, 2] = img[0, 0, :, :] + MEAN_VALUES[0]
   
    return result

In [53]:
data = []

for img_path in [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(images_path)) for f in fn]:
    data.append(features(preprocess(proportional_resize(imread(img_path)))))
    break